Ask_media via pinecone
- set request & filters
- embed request
- ask pinecone - embed & filters -> get texts & links

# Imports & constants

In [1]:
import pinecone
import json
from tqdm.notebook import tqdm_notebook
from tqdm import tqdm
import os
import pandas as pd
import datetime
from datetime import timedelta, datetime
import time

import openai

keys_path = '../keys/'
data_path = '../../TG_messages/'

/Users/alexbadin/miniconda3/envs/ask_media/lib/python3.10/site-packages/pinecone/index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
with open(keys_path+'api_keys.json') as f:
  data = json.loads(f.read())

#load openai credentials
openai_key = data['openai_key']

# load pinecone credentials
pine_key = data['pine_key']
pine_env = data['pine_env']

In [3]:
pinecone.init(api_key=pine_key, environment=pine_env)
index_name = 'tg-news'

index = pinecone.Index(index_name)

index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.4712,
 'namespaces': {'': {'vector_count': 47120}},
 'total_vector_count': 47120}

In [7]:
# set up openai
openai.api_key = openai_key
def get_embedding(text, model="text-embedding-ada-002"):
   text = text.replace("\n", " ")
   return openai.Embedding.create(input = [text], model=model)['data'][0]['embedding']

# Ask pinecone

In [5]:
# FILTERS
# metadata: 'cleaned_message', 'summary', 'stance', 'channel', 'date', 'views'
stance = 'voenkor'
dates = ['2021-10-20', '2021-10-24']
# define start and end dates (if end date is not defined, it will be set to today)
if len(dates) == 2:
  start_date = dates[0]
  end_date = dates[1]
else:
  start_date = dates[0]
  end_date = datetime.today().strftime('%Y-%m-%d')

# convert start_date to int
start_date = int(datetime.strptime(start_date, '%Y-%m-%d').timestamp())
end_date = int(datetime.strptime(end_date, '%Y-%m-%d').timestamp())

# filtering
filter = {
  "stance": { "$eq": stance },
  "date": { "$gte": start_date },
  "date": { "$lte": end_date }
}

In [10]:
filter

{'stance': {'$eq': 'voenkor'}, 'date': {'$lte': 1635019200}}

In [16]:
dates = ['2021-10-20']

if dates:
    if len(dates) == 2:
        # convert start_date to int
        start_date = dates[0]
        end_date = dates[1]
    else:
        start_date = dates[0]
        end_date = datetime.today().strftime('%Y-%m-%d')
else:
    start_date = '2022-02-01'
    end_date = datetime.today().strftime('%Y-%m-%d')

# filtering
start_date = int(datetime.strptime(start_date, '%Y-%m-%d').timestamp())
end_date = int(datetime.strptime(end_date, '%Y-%m-%d').timestamp())

filter = {
    "stance": { "$eq": stance },
    "date": { "$gte": start_date, "$lte": end_date },
    }

In [17]:
filter

{'stance': {'$eq': 'voenkor'},
 'date': {'$gte': 1634673600, '$lte': 1698004800}}

In [18]:
question_emb = get_embedding("Что происходит в Израиле")
res = index.query(question_emb, top_k=10, include_metadata=True, filter=filter)

In [28]:
res.to_dict()['matches']

[{'id': 'voenacher_54287',
  'score': 0.865285635,
  'values': [],
  'metadata': {'channel': 'voenacher',
   'cleaned_message': '❗ Разборы Рыбаря: итоги 12 октября 2023 года\n\nМихаил Звинчук в своей авторской передаче на @SolovievLive подводит итоги событий на фронтах специальной военной операции и рассказывает о значимых происшествиях в кризисных районах мира.\n\nВ сегодняшнем выпуске:\n\n Зона израильско-палестинского конфликта:\n▪продолжение ударов по городам Израиля и бомбардировок Сектора Газа.\n▪Египет готовит гуманитарную миссию;\n▪США перебрасывают спецназ для эвакуации заложников;\n▪глава Госдепа Энтони Блинкен посетил в Израиль;\n▪зачем ВВС Израиля опять ударили по Сирии;\n▪как эскалация отразилась на экономике Израиля.\n\n Зона специальной военной операции:\n▪продолжение боев на Авдеевском участке;\n▪временное затишье на Времьевском выступе.\n\nПостсоветское пространство:\n▪Молдавия окончательно теряет единственный морской порт.\n#видео #разбор\n@rybar\n\nПоддержать нас',
 

In [74]:
# add criteria to filter dict
filter = {}
if stance:
    filter['stance'] = {"$eq": stance}
if dates:
    if len(dates) == 2:
        start_date = int(datetime.strptime(dates[0], '%Y-%m-%d').timestamp())
        end_date = int(datetime.strptime(dates[1], '%Y-%m-%d').timestamp())
        filter['&and'] = [{"date": {"$gte": start_date}}, {"date": {"$lte": end_date}}]
    else:
        filter['date'] = {"$gte": start_date}

In [81]:
if dates:
    if len(dates) == 2:
        # convert start_date to int
        start_date = int(datetime.strptime(dates[0], '%Y-%m-%d').timestamp())
        end_date = int(datetime.strptime(dates[1], '%Y-%m-%d').timestamp())
    else:
        start_date = dates[0]
        end_date = datetime.today().strftime('%Y-%m-%d')
else:
    start_date = int(datetime.strptime('2022-02-01', '%Y-%m-%d').timestamp())
    end_date = datetime.today().strftime('%Y-%m-%d')

# keep filter as json
filter = json.dumps(filter)

filter = {
    "stance": { "$eq": stance },
    "date": { "$gte": start_date },
    "date": { "$lte": end_date }
    }

In [70]:
dates

['2023-10-02', '2023-10-03']

In [82]:
filter

{'stance': {'$eq': 'voenkor'}, 'date': {'$lte': 1696276800}}

In [105]:
start_date = datetime.today().strftime('%Y-%m-%d')
int(datetime.strptime(start_date, '%Y-%m-%d').timestamp())


1697745600

In [ ]:
# FILTERS
# metadata: 'cleaned_message', 'summary', 'stance', 'channel', 'date', 'views'
stance = 'voenkor'

dates = ['2023-10-02', '2023-10-03']

if dates:
    if len(dates) == 1:
        filter = {
            "stance": { "$eq": stance },
            "date": { "$gte": start_date }
            }
    elif len(dates) == 2:
        filter = {
            "stance": { "$eq": stance },
            "date": { "$gte": start_date },
            "date": { "$lte": end_date }
            }

In [76]:
question_emb = get_embedding("Как делa на фронте?")
res = index.query(question_emb, top_k=10, include_metadata=True, filter=filter)
res.to_dict().keys()

ApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'content-type': 'application/json', 'Content-Length': '145', 'date': 'Fri, 20 Oct 2023 18:06:55 GMT', 'x-envoy-upstream-service-time': '2', 'server': 'envoy', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"code":3,"message":"illegal condition for field &and, got [{\"date\":{\"$gte\":1696190400.0}},{\"date\":{\"$lte\":1696276800.0}}]","details":[]}


In [37]:
# pd.DataFrame(res.to_dict()['matches'])
# convert metadata column to dataframe
# pd.DataFrame(res.to_dict()['matches'])['metadata'].apply(pd.Series)
# add metadata to dataframe
df = pd.DataFrame(res.to_dict()['matches']).join(pd.DataFrame(res.to_dict()['matches'])['metadata'].apply(pd.Series))
df[['id', 'score', 'cleaned_message']]

,id,score,cleaned_message
0,voenacher_53307,0.864290,Хотели красивое?\nИх есть у меня!\n\nЗапорожск...
1,sashakots_42547,0.837429,"Друзья, мы с вами долго собирали помощь отдель..."
2,rybar_52689,0.833296,❗ Хроника специальной военной операции \nза 4 ...
3,rybar_52688,0.832837,❗ Разборы Рыбаря: итоги 4 октября 2023 года\n\...
4,voenacher_53413,0.832837,❗ Разборы Рыбаря: итоги 4 октября 2023 года\n\...
5,voenacher_53378,0.832678,Комментарий для ТАСС о применении наших FPV-др...
6,rybar_52587,0.832016,События дня на фронтах СВО из-под кисти наших...
7,voenacher_53252,0.830576,❗ Разборы Рыбаря: итоги 30 сентября - 1 октябр...
8,rybar_52581,0.830462,❗ Разборы Рыбаря: итоги 30 сентября - 1 октябр...
9,rybar_52686,0.827612,Через 15 минут на @SolovievLive выйдет авторс...


In [32]:
start_date, end_date

(1696190400, 1696363200)

In [36]:
df['date'].min(), df['date'].max()

(1696104000.0, 1696363200.0)

In [60]:
i = 4
# print wrapped text
import textwrap
print('\n'.join(textwrap.wrap(df.summary.to_list()[i], 80)))

Михаил Звинчук в своей авторской передаче на @SolovievLive подводит итоги
событий на фронтах специальной военной операции и рассказывает о значимых
происшествиях в кризисных районах мира. Зона специальной военной операции:
▪успешные контратаки ВС РФ под Урожайным и Малиновкой на Времьевском участке
фронта; ▪комбинированная атака на Крым и проваленная высадка десанта украинского
ГУР на мыс Тарханкут; ▪иранские снаряды в зоне СВО: почему российские РСЗО бьют
«Арашами». Закавказье: ▪за сколько продали Арцах: доступ к каким ресурсам
получил Запад; ▪в Армении одобрили Римский статут: в чем риски для России;
▪логистические проекты Турции на Южном Кавказе.


In [62]:
print(df.cleaned_message.to_list()[i])

❗ Разборы Рыбаря: итоги 4 октября 2023 года

Михаил Звинчук в своей авторской передаче на @SolovievLive подводит итоги событий на фронтах специальной военной операции и рассказывает о значимых происшествиях в кризисных районах мира.

В сегодняшнем выпуске:

 Зона специальной военной операции:
▪успешные контратаки ВС РФ под Урожайным и Малиновкой на Времьевском участке фронта;
▪комбинированная атака на Крым и проваленная высадка десанта украинского ГУР на мыс Тарханкут;
▪иранские снаряды в зоне СВО: почему российские РСЗО бьют «Арашами».

 Европа и соседи:
▪поляки ремонтируют «Леопарды» для нужд украинских формирований;
▪в Венгрии, Молдавии и Румынии требуют прекратить мобилизацию меньшинств на Украине;
▪планы Чехии по запуску производства винтовок и дальнейшие поставки техники в зону конфликта.

Закавказье:
▪за сколько продали Арцах: доступ к каким ресурсам получил Запад;
▪в Армении одобрили Римский статут: в чем риски для России;
▪логистические проекты Турции на Южном Кавказе.

 Ближн

- make link
- take sim score
- take text

In [48]:
# get top relevant news (TO BE CHANGED)
def get_top_openai(request=None, request_emb=None, dates=None, stance=None, model="text-embedding-ada-002", top_n=10):
    dates=dates
    stance=stance
    
    if request_emb is None and request is None:
        print('Error: no request')
        return
    if request_emb is None:
        request_emb = get_embedding(request)
    
    # filtering
    if dates:
        if len(dates) == 1:
            filter = {
                "stance": { "$eq": stance },
                "date": { "$gte": start_date }
                }
        elif len(dates) == 2:
            filter = {
                "stance": { "$eq": stance },
                "date": { "$gte": start_date },
                "date": { "$lte": end_date }
                }
    
    # query pinecone
    res = index.query(question_emb, top_k=10, include_metadata=True, filter=filter)
    top_sim_news = pd.DataFrame(res.to_dict()['matches']).join(pd.DataFrame(res.to_dict()['matches'])['metadata'].apply(pd.Series))
    
    # collect links & similarities
    top_sim_news['msg_id'] = top_sim_news['id'].apply(lambda x: x.split('_')[-1])
    top_sim_news['channel_name'] = top_sim_news['id'].apply(lambda x: '_'.join(x.split('_')[:-1]))

    top_sim_news['link'] = top_sim_news.apply(lambda x: "https://t.me/"+str(x.channel_name)+"/"+str(x.msg_id)+" - "+str(round(x.score,3)), axis=1)
    news_links = '\n'.join(top_sim_news['link']tolist())
    # collect news
    news4request = '\n'.join(top_sim_news['summary'].tolist())
    return news4request, news_links

In [49]:
request="Как делa на фронте?"
stance = 'voenkor'
dates = ['2023-10-02', '2023-10-03']

news4request, news_links = get_top_openai(request=request, stance = stance, dates = dates)

In [53]:
print(news4request)

Хотели красивое?
Их есть у меня!

Запорожский фронт.
Друзья, мы с вами долго собирали помощь отдельному разведбатальону, который действует на Запорожском направлении. И наконец собрали. На днях большой груз в рамках проекта Народного фронта «Все для Победы» был доставлен адресатам. Обзор и отчет чуть позже опубликую.

Не успели мы вручить им нужные девайсы, как они начали приносить пользу. В первые же сутки парни заземлили новеньким противодронным ружьем FPV-дрон с подарунком. Никто не пострадал. 

Спасибо всем, принявшим участие в сборе! Все для фронта, все для Победы!

@sashakots
❗ Хроника специальной военной операции 
за 4 октября 2023 года

Ночью ВСУ в очередной раз атаковали Крым, применив противокорабельные ракеты. Снаряды удалось перехватить, однако через несколько часов противник попытался высадить десант у мыса Прибойный. Российские пограничники перехватили ДРГ и взяли одного из диверсантов в плен.

Также ночью ВСУ предприняли массовую атаку беспилотников на приграничные район

In [ ]:
## (CHANGE: ONLY IN HOW TO GET get_top_openai)
# FUNCTION ask_media to combine all together (TO USE IN TG BOT REQUESTS)
def ask_media(request, dates=None, sources=None, stance=None, model_name = "gpt-3.5-turbo", tokens_out = 512, full_reply = True):
    # check request time
    request_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())

    # get top news
    # INPUT: request, dates, sources, stance
    # OUTPUT: news4request - list of news texts for openai, news_links - list of links
    news4request, news_links = get_top_openai(request, dates=dates, sources=sources, stance=stance, model="text-embedding-ada-002", top_n=10)
    # limit number of tokens vs model
    if model_name == "gpt-3.5-turbo":
        # print(type(news4request), len(news4request))
        # print(news4request)
        news4request = news4request[:4000]
    elif model_name == "gpt-3.5-turbo-16k":
        news4request = news4request[:16000]
    elif model_name == "gpt-4":
        news4request = news4request[:8000]
    elif model_name == "gpt-4-32k":
        news4request = news4request[:32000]
    
    reply = ask_openai(request, news4request, model_name = model_name, tokens_out = tokens_out)
    request_params = f"Request: {request}; \nFilters: dates: {dates}; sources: {sources}; stance: {stance}"
    reply_text = reply.choices[0]['message']['content']
    n_tokens_used = reply.usage.total_tokens
    reply_cost = n_tokens_used / 1000 * price_1K

    # write params & reply to file. If file doesn't exist - create it with headers
    # check reply time
    reply_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
    
    if not os.path.isfile('openai_chatbot.csv'):
        with open('openai_chatbot.csv', 'a', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(['request', 'dates', 'sources', 'stance', 'reply_text', 'reply_cost', 'request_time', 'reply_time', 'model_name', 'n_tokens_used', 'news_links'])
    with open('openai_chatbot.csv', 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow([request, dates, sources, stance, reply_text, reply_cost, request_time, reply_time, model_name, n_tokens_used, news_links])
    
    # return reply for chatbot. If full_reply = False - return only reply_text
    if full_reply == False:
        return reply_text
    else:
        return request_params + "\n" + "Cost per request: " + str(round(reply_cost,3)) + ". Tokens used: " + str(n_tokens_used) + "\n\n" + reply_text + "\n\n" + news_links

In [ ]:
## (NOT CHANGED)
## Ask OpenAI 
def ask_openai(request, news4request, model_name = "gpt-3.5-turbo", tokens_out = 512):

    system_content_en = f"You are given few short news texts in Russian. Based on these texts you need to answer the following question: {request}. \
        First, analyze if the texts provide an answer to the question. \
        If the texts do not provide proper answer, say that. \
        If they do, select the texts relevant to the question ({request}) and summarize them. \
        \nОтвечай только на русском. Не более 1000 символов."

    system_content_ru = f"Тебе будут представлены несколько новостей. На их основе нужно ответить на вопрос: {request}. \
        Сперрва проверь, что новости содержат ответ. \
        Если ответа в новостях нет, так и ответь. \
        Далее, отбери новости, которые отвечают на вопрос ({request}) и сделай но ним резюме. \
        \nНе более 1000 символов."

    response = openai.ChatCompletion.create(
        model = model_name,
        messages=[
            {
            "role": "system",
            "content": system_content_ru
            },
            {
            "role": "user",
            "content": news4request
            }
        ],
        temperature=0,
        max_tokens=tokens_out,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
        )
    return response